In [ ]:
import os
import numpy as np
import torch
from collections import OrderedDict
import json
from models.mlp import MLP_MoCo
from tqdm import tqdm

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

In [ ]:
query_feature_A_dir = '/nfs3-p2/zsxm/naic/preliminary/test_A/query_feature_A'
gallery_feature_A_dir = '/nfs3-p2/zsxm/naic/preliminary/test_A/gallery_feature_A'

In [ ]:
query_feature_A = []
for data in sorted(os.listdir(query_feature_A_dir)):
    query_feature_A.append(torch.from_numpy(np.fromfile(os.path.join(query_feature_A_dir, data), dtype='<f4')))
query_feature_A = torch.stack(query_feature_A).to(device)

In [ ]:
gallery_feature_A = []
for data in sorted(os.listdir(gallery_feature_A_dir)):
    gallery_feature_A.append(torch.from_numpy(np.fromfile(os.path.join(gallery_feature_A_dir, data), dtype='<f4')))
gallery_feature_A = torch.stack(gallery_feature_A).to(device)

In [ ]:
print(query_feature_A.shape, query_feature_A)
print(gallery_feature_A.shape, gallery_feature_A)
torch.save(query_feature_A.cpu(), '/nfs3-p2/zsxm/naic/preliminary/test_A/query_feature_A.pt')
torch.save(gallery_feature_A.cpu(), '/nfs3-p2/zsxm/naic/preliminary/test_A/gallery_feature_A.pt')

In [ ]:
query_feature_A = torch.load('/nfs3-p2/zsxm/naic/preliminary/test_A/query_feature_A.pt')
gallery_feature_A = torch.load('/nfs3-p2/zsxm/naic/preliminary/test_A/gallery_feature_A.pt')
print(query_feature_A.shape, query_feature_A)
print(gallery_feature_A.shape, gallery_feature_A)

In [ ]:
query_feature_A = query_feature_A.to(device)
gallery_feature_A = gallery_feature_A.to(device)

In [ ]:
def cos_similarity(q, k):
    try:
        mm = torch.mm(q, k.T)
        qn = torch.linalg.vector_norm(q, dim=1, keepdim=True)
        kn = torch.linalg.vector_norm(k, dim=1, keepdim=True)
        qk = torch.mm(qn, kn.T)
        res = mm / qk
    except RuntimeError:
        q = q.cpu()
        k = k.cpu()
        mm = torch.mm(q, k.T)
        qn = torch.linalg.vector_norm(q, dim=1, keepdim=True)
        kn = torch.linalg.vector_norm(k, dim=1, keepdim=True)
        qk = torch.mm(qn, kn.T)
        res = mm / qk
    return res

In [ ]:
def cos_similarity(q, k):
    try:
        q = torch.nn.functional.normalize(q, dim=1)
        k = torch.nn.functional.normalize(k, dim=1)
        res = torch.mm(q, k.T)
    except RuntimeError:
        q = q.cpu()
        k = k.cpu()
        q = torch.nn.functional.normalize(q, dim=1)
        k = torch.nn.functional.normalize(k, dim=1)
        res = torch.mm(q, k.T)
    return res

In [ ]:
@torch.no_grad()
def batch_cos(q, k, batch_size=2048):
    res = []
    for i in tqdm(range(0, q.shape[0], batch_size)):
        bres = cos_similarity(q[i:i+batch_size], k)
        res.append(bres)
    return torch.cat(res)

In [ ]:
res = batch_cos(query_feature_A, gallery_feature_A)
print(res.shape)

In [ ]:
net = MLP_MoCo()
net.load_state_dict(torch.load('.details/checkpoints/MLP/01-11_11:48:10/Net_best.pth', map_location=device))
net.to(device)
net = net.encoder_q#.encoder
net.eval()
print('')

In [ ]:
@torch.no_grad()
def encode(net, data, batch_size=2048):
    res, rcons = [], []
    for i in tqdm(range(0, data.shape[0], batch_size)):
        brcons, bres = net(data[i:i+batch_size])
        res.append(bres)
        rcons.append(brcons)
    return torch.cat(res), torch.cat(rcons)

In [ ]:
query_code_A, query_recons_A = encode(net, query_feature_A)
gallery_code_A, gallery_recons_A = encode(net, gallery_feature_A)

In [ ]:
print(query_code_A.shape, query_recons_A.shape)
print(gallery_code_A.shape, gallery_recons_A.shape)

In [ ]:
res2 = batch_cos(query_code_A, gallery_code_A)
print(res.shape)

In [ ]:
def print_res(res, print_json=False):
    res_dict = {}
    query_names = sorted(os.listdir(query_feature_A_dir))
    gallery_names = sorted(os.listdir(gallery_feature_A_dir))
    for i, name in enumerate(tqdm(query_names)):
        idx = torch.argsort(res[i], dim=-1, descending=True)
        query_res = []
        for j in range(100):
            query_res.append(gallery_names[idx[j]])
        res_dict[name] = query_res
    if print_json:
        with open('./sub_a.json', 'w') as f:
        json.dump(res_dict, f)

In [ ]:
def compare_res(res1, res2, topk=100):
    assert res1.shape[0] == res2.shape[0]
    ave = 0
    t = tqdm(range(res1.shape[0]))
    for i in t:
        idx1 = set(torch.argsort(res1[i], dim=-1, descending=True)[:topk].tolist())
        idx2 = set(torch.argsort(res2[i], dim=-1, descending=True)[:topk].tolist())
        iou = len(idx1&idx2)/len(idx1|idx2)
        t.set_postfix(IoU=f'{iou:.4f}')
        ave += iou
    ave /= res1.shape[0]
    return ave

In [ ]:
ave = compare_res(res, res2)
print(ave)

## 马氏距离计算

In [ ]:
qfs = query_feature_A.sum(dim=0)
nqfs = torch.arange(2048)[qfs!=0]
print(nqfs.shape, nqfs)
gfs = gallery_feature_A.sum(dim=0)
ngfs = torch.arange(2048)[gfs!=0]
print(ngfs.shape, ngfs)
print(nqfs.equal(ngfs))

In [ ]:
query_reshape_A = query_feature_A[:, qfs!=0]
gallery_reshape_A = gallery_feature_A[:, gfs!=0]
print(query_reshape_A.shape, gallery_reshape_A.shape)

In [ ]:
del query_feature_A, gallery_feature_A

In [ ]:
def Mahalanobis(q, k):
    query_names = sorted(os.listdir(query_feature_A_dir))
    gallery_names = sorted(os.listdir(gallery_feature_A_dir))
    q = torch.nn.functional.normalize(q, dim=1)
    k = torch.nn.functional.normalize(k, dim=1)
    
    mean_k = k.mean(dim=0, keepdim=True)
    sk = k - mean_k
    cov = torch.mm(sk.T, sk)/(sk.shape[0]-1)
    print(cov.shape, cov)
    cov = cov.to(torch.device('cpu'))
    icov = torch.linalg.inv(cov)
    print(icov.shape, icov)
    del cov
    icov = icov.to(device)
    res_dict = {}
    for i in tqdm(range(q.shape[0])):
        sub = q[i] - k
        mul = torch.mm(sub, icov)
        r = torch.einsum('bi,bi->b', mul, sub)
        assert r.shape == (k.shape[0],)
        idx = torch.argsort(r, descending=False)
        query_res = []
        for j in range(100):
            query_res.append(gallery_names[idx[j]])
        res_dict[query_names[i]] = query_res
    return res_dict

In [ ]:
res3 = Mahalanobis(query_reshape_A, gallery_reshape_A)

In [ ]:
with open('./sub_a.json', 'w') as f:
    json.dump(res3, f)

# ReRanking

In [1]:
import os
import numpy as np
from collections import OrderedDict
import json
from tqdm import tqdm

query_feature_A_dir = '/nfs3-p2/zsxm/naic/preliminary/test_A/query_feature_A'
gallery_feature_A_dir = '/nfs3-p2/zsxm/naic/preliminary/test_A/gallery_feature_A'

In [ ]:
query_feature_A = []
for data in sorted(os.listdir(query_feature_A_dir)):
    query_feature_A.append(np.fromfile(os.path.join(query_feature_A_dir, data), dtype='<f4'))
query_feature_A = np.stack(query_feature_A)
gallery_feature_A = []
for data in sorted(os.listdir(gallery_feature_A_dir)):
    gallery_feature_A.append(np.fromfile(os.path.join(gallery_feature_A_dir, data), dtype='<f4'))
gallery_feature_A = np.stack(gallery_feature_A)
print(query_feature_A.shape, gallery_feature_A.shape)
qfs = query_feature_A.sum(axis=0)
nqfs = np.arange(2048)[qfs!=0]
print(nqfs.shape, nqfs)
gfs = gallery_feature_A.sum(axis=0)
ngfs = np.arange(2048)[gfs!=0]
print(ngfs.shape, ngfs)
print((nqfs==ngfs).all)
query_reshape_A = query_feature_A[:, qfs!=0]
gallery_reshape_A = gallery_feature_A[:, gfs!=0]
print(query_reshape_A.shape, gallery_reshape_A.shape)
np.save('/nfs3-p2/zsxm/naic/preliminary/test_A/query_feature_A.npy', query_feature_A)
np.save('/nfs3-p2/zsxm/naic/preliminary/test_A/gallery_feature_A.npy', gallery_feature_A)
np.save('/nfs3-p2/zsxm/naic/preliminary/test_A/query_reshape_A.npy', query_reshape_A)
np.save('/nfs3-p2/zsxm/naic/preliminary/test_A/gallery_reshape_A.npy', gallery_reshape_A)

In [2]:
query_reshape_A = np.load('/nfs3-p2/zsxm/naic/preliminary/test_A/query_reshape_A.npy')
gallery_reshape_A = np.load('/nfs3-p2/zsxm/naic/preliminary/test_A/gallery_reshape_A.npy')

In [3]:
from re_ranking.re_ranking_feature import re_ranking

In [6]:
re_ranking(query_reshape_A, gallery_reshape_A, 100, 30, 0.3, MemorySave=True, Minibatch=200)

computing original distance


MemoryError: Unable to allocate 375. GiB for an array with shape (448794, 448794) and data type float16

In [ ]:
from datasets.preliminary_dataset import PreliminaryDataset, PreliminaryBatchSampler, preliminary_collate_fn
import random
from torch.utils.data import DataLoader

In [ ]:
dataset = PreliminaryDataset('/nfs3-p1/zsxm/naic/preliminary/train', False)

In [ ]:
batchsampler = PreliminaryBatchSampler(dataset, 150)

In [ ]:
dataloader = DataLoader(dataset, batch_sampler=batchsampler, num_workers=8, collate_fn=preliminary_collate_fn, pin_memory=True)

In [ ]:
random.seed(2)#2078
test_count = 0
while True:
    batchs = []
    t_batch = []
    for i, b in enumerate(batchsampler):
        #print(i, b)
        batchs.append(b)
        t_batch.extend(b)
        b_len = 0
        for idx in b:
            b_len += dataset.idx2len[idx]
        assert 0 < b_len <= batchsampler.batch_size, str(b_len)+str(b)+str(i)
    assert len(t_batch) == 15000, len(t_batch)
    t_batch_set = set(t_batch)
    assert len(t_batch) == len(t_batch_set), len(t_batch_set)
    test_count +=1
    print(test_count)

In [ ]:
count = 0
for q, k, q_label, k_label in dataloader:
    print(count)
    print(q.shape)
    print(k.shape)
    print(q_label.shape)
    print(k_label.shape)
    count += 1

In [ ]:
k_label